# 1.0 Arrays

## 1.1 Get product of all other elements

### Problem Statement
Given an array of integers, return a new array such that each element at index $i$ in the new array is equal to the product of all but the ith element.  

Use of division is prohibited.

In [1]:
import collections
import unittest


def products(elems):
    """Return an array containing the n-1 products of the input."""
    assert not(len(elems) < 3), 'invalid: len(elems) < 3'

    forward, backward = elems[:], elems[:]

    # Obtain products in the forward direction.
    for ind in range(1,len(elems)):
        forward[ind] = forward[ind]*forward[ind-1]

    # Obtain products in the backward direction.
    for ind in range(len(elems)-2,-1,-1):
        backward[ind] = backward[ind]*backward[ind+1]

    # Compute each of the n-1 products.
    products = [None]*len(elems)
    products[0], products[-1] = backward[1], forward[-2]
    for ind in range(1,len(elems)-1):
        products[ind] = forward[ind-1]*backward[ind+1]

    return products


class ProductsTest(unittest.TestCase):
    
    def test_products(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            # 1..5
            case([1,2,3,4,5], [120,60,40,30,24]),
            # 5..1
            case([5,4,3,2,1], [24,30,40,60,120]),
            # Non-contiguous values.
            case([6,9,2,5,3], [270,180,810,324,540]),
            # Mix of negative and positive values.
            case([-5,10,2,-4], [-80,40,200,-100]),
        ]
        for c in cases:
            rcv = products(c.input)
            self.assertEqual(rcv, c.expected)


unittest.main(ProductsTest(), argv=[''], verbosity=2, exit=False)

test_products (__main__.ProductsTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## 1.2 Locate smallest window to be sorted

### Problem Statement
Given an array of integers that are out of order, determine the indices of the smallest window that must be sorted in order for the entire array to be sorted.

In [2]:
import collections
import unittest


def smallest_unsorted_window(elems):
    """Return indices of the smallest unsorted window in the input."""
    assert not(len(elems) < 2), 'invalid: len(elems) < 2'

    unsorted = False

    # Scan left-to-right searching for rightmost unsorted element.
    max_value, max_ind = elems[0], 0
    for ind in range(1, len(elems)):
        max_value = max(max_value, elems[ind])
        if elems[ind] < max_value:
            max_ind, unsorted = ind, True 
    
    # Scan right-to-left searching for leftmost unsorted element.
    min_value, min_ind = elems[-1], len(elems)-1
    for ind in range(len(elems)-1,-1,-1):
        min_value = min(min_value, elems[ind])
        if elems[ind] > min_value:
            min_ind, unsorted = ind, True

    return [min_ind, max_ind] if unsorted else [None, None]


class SmallestUnsortedWindowTest(unittest.TestCase):
    
    def test_smallest_unsorted_window(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            # Sorted.
            case([3,5,6,7,9], [None,None]),
            # Reversed.
            case([9,7,6,5,3], [0,4]),
            # Permutations of the same unsorted input.
            case([3,7,5,6,9], [1,3]),
            case([3,6,5,7,9], [1,2]),
            case([7,3,5,6,9], [0,3]),
            case([9,3,5,6,7], [0,4]),
            case([5,3,6,7,9], [0,1]),
            case([3,5,6,9,7], [3,4]),
            case([5,9,7,6,3], [0,4]),
        ]
        for c in cases:
            rcv = smallest_unsorted_window(c.input)
            self.assertEqual(rcv, c.expected)


unittest.main(SmallestUnsortedWindowTest(), argv=[''], verbosity=2, 
              exit=False)

test_smallest_unsorted_window (__main__.SmallestUnsortedWindowTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.011s

OK


## 1.3 Calculate maximum subarray sum

### Problem Statement
Given an array of integers, find the maximum subarray sum in the input.

In [3]:
import collections
import unittest


def max_subarray_sum(elems):
    """Return the maximum subarray sum of the input."""
    max_sum, curr_sum = 0, 0
    for x in elems:
        # Update the max, handle edge case for last element on return.
        max_sum = max(max_sum, curr_sum)
        if curr_sum + x > 0:
            curr_sum += x  # Add to existing sum.
        else:
            curr_sum = 0  # Start a new sum.
    return max(max_sum, curr_sum)


class MaxSubarraySumTest(unittest.TestCase):
    
    def test_max_subarray_sum(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            # All negative.
            case([-1,-2,-3], 0),
            # Random inputs.
            case([34,-50,42,14,-5,86], 137),
            case([-10,2,3,-2,0,5,-15], 8),
            case([2,3,-2,-1,10], 12),
            case([20,30,-70,40,20], 60),
            case([-70,20,30,40,20], 110),
            case([20,30,40,20,-70], 110),
            case([20,-70,30,40,-10], 70),
        ]
        for c in cases:
            rcv = max_subarray_sum(c.input)
            self.assertEqual(rcv, c.expected)


unittest.main(MaxSubarraySumTest(), argv=[''], verbosity=2, exit=False)

test_max_subarray_sum (__main__.MaxSubarraySumTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.014s

OK


## 1.4 Find number of smaller elements to the right

### Problem Statement
Given an array of integers, return an array containing the count of elements to the right of each element which are smaller in value.

In [4]:
import collections
import heapq
import unittest


def count_smaller(elems):
    """Return the count of smaller elements to right of each element."""
    assert not(len(elems) < 1), 'invalid: len(elems) < 1'
        
    # Iterate right-to-left adding each element to min heap.
    #
    # If the item added to the heap is not the minimum, then
    # count the number of elements which are smaller by successively
    # popping the minimum element from the heap.
    counts, minheap = [0]*len(elems), [elems[-1]]
    for ind in range(len(elems)-2,-1,-1):
        if elems[ind] > minheap[0]:
            stack = collections.deque()
            while True:
                stack.appendleft(heapq.heappop(minheap))
                if len(minheap) < 1 or elems[ind] <= minheap[0]:
                    break
            counts[ind] = len(stack)
            # Put the popped elements back on the heap.
            while len(stack) > 0:
                heapq.heappush(minheap, stack.popleft())
        # Add the current element to the heap.
        heapq.heappush(minheap, elems[ind])                    

    return counts


class CountSmallerTest(unittest.TestCase):
    
    def test_count_smaller(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            # Sorted.
            case([1,2,3,4,5], [0,0,0,0,0]),
            case([5,4,3,2,1], [4,3,2,1,0]),
            # Random inputs.
            case([3,4,9,6,1], [1,1,2,1,0]),
            case([3,7,5,4,8,6], [0,3,1,0,1,0]),
        ]
        for c in cases:
            rcv = count_smaller(c.input)
            self.assertEqual(rcv, c.expected)


unittest.main(CountSmallerTest(), argv=[''], verbosity=2, exit=False)

test_count_smaller (__main__.CountSmallerTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
